In this capstone project, I used agent-based modeling in Python to simulate human patterns of movement within a transportation system, focusing on mode choice between private vehicles and public transit. I added a coefficient of bias to investigate the emergent behavior of the system when transit mode choices were influenced by the proportions of other riders, finding that interpersonal bias in this model can exacerbate exisiting disparities between groups of travelers.

The full report on this project can be read [here](). The following is a simplified version of the code used to generate my results.

## Code


In [ ]:
from numpy import *
from numpy.random import choice
from random import randint, choice


### Initialize

# matrix of connections
Network = array([[1, 1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0, 0],
          [0, 0, 1, 1, 1, 1, 0, 0],
          [0, 0, 0, 1, 1, 1, 1, 0],
          [0, 0, 0, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 1, 1, 1]])
# matrix of relative appeal
high = 0.8
med = 0.5
low = 0.2
relApp = array([[0, low, med, 0, 0, 0, 0, 0],
          [low, 0, med, 0, 0, 0, 0, 0],
          [med, med, 0, high, 0, 0, 0, 0],
          [0, 0, high, 0, high, med, 0, 0],
          [0, 0, 0, high, 0, med, low, 0],
          [0, 0, 0, med, med, 0, low, med],
          [0, 0, 0, 0, low, low, 0, low],
          [0, 0, 0, 0, 0, med, low, 0]]) #remember matrices are 0-7

class person:
    def __init__(self):
        self.location = 0 # later randomize 0-7
        self.affiliation = 0 # later determine -1 or 1
        self.appeal = 0
        self.comfort = 0 # later randomize, sum to 1

The following function runs a model with strong bias interactions on the part of both groups. It takes as input a network and relative appeals matrix defined above, a maximum amount of timesteps, a number of agents, an amount of destinations, and a "bar" which defines the borders between the neighborhoods, as seen in the title image and described in the report.


In [ ]:
def transitMove(network, relapp, tmax = 10, n = 10000, m = 8, bar = 4):
    # empty matrices of travelers
    locations = zeros((m,tmax))
    transit_1 = zeros((m ,m, tmax))
    transit_1[:,:,0] = ones((m,m))
    transit_2 = zeros((m, m, tmax))
    transit_2[:,:,0] = ones((m,m))
    cars_1 = zeros((m, m, tmax))
    cars_1[:,:,0] = ones((m,m))
    cars_2 = zeros((m, m, tmax))
    cars_2[:,:,0] = ones((m,m))
            
    # randomize people
    people = [person() for j in range(n)]

    for i in people:
        i.location = randint(0,m - 1)
        locations[i.location,0] +=1
        if (i.location <bar):
            i.affiliation = 1
        else:
            i.affiliation = 2
        appeal = randint(1,100)
        comfort = randint(1,100)
        total = appeal + comfort
        i.appeal = appeal/total
        i.comfort = comfort/total
        
    # run the model
    comfortCoeff1 = zeros((m,m,tmax))
    comfortCoeff2 = zeros((m,m,tmax))
    transitTravTotal = zeros((m,m))
    for t in range(tmax-1):
        # create matrix of comfort coefficients
        transitTravTotal = transitTravTotal + transit_1[:,:,t] + transit_2[:,:,t] # total travelers over all of human history
        comfortCoeff1[:,:,t] = sum(transit_1[:,:,0:(t+1)], axis = 2)/transitTravTotal 
        comfortCoeff2[:,:,t] = sum(transit_2[:,:,0:(t+1)], axis = 2)/transitTravTotal 
        for i in people:
            # select destination
            row = network[i.location]
            indices = where(row == 1)
            destination = choice(indices[0])
            if (destination == i.location):
                locations[i.location,t+1] +=1
            else:
                # calculate preference
                if (i.affiliation ==1):
                  preference = i.appeal * relapp[i.location,destination]+ i.comfort * comfortCoeff1[i.location, destination, t]
              else:
                  preference = i.appeal * relapp[i.location,destination]+ i.comfort * comfortCoeff2[i.location, destination, t]
                # choose mode
                choiceVar = randint(0,100)/100
                if (preference > choiceVar):
                    # they choose transit
                    if (i.affiliation ==1):
                        transit_1[i.location, destination, t+1] += 1
                    else:
                        transit_2[i.location, destination, t+1] += 1
                else:
                    # they choose car
                    if (i.affiliation ==1):
                        cars_1[i.location, destination, t+1] += 1
                    else:
                        cars_2[i.location, destination, t+1] += 1
                # update location
                i.location = destination
                locations[i.location,t+1] +=1
            # end if statement
        # end for loop
    results = {"transit_1": transit_1, "transit_2": transit_2, "cars_1": cars_1,"cars_2": cars_2, "locations": locations}
    return(results)

The following function graphs some of the outputs of this model in a way that compares transit adoption percentages for the two groups.


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_pop_compare(transit_1, transit_2, locations, title = False, n = 10000, tmax = 10, bar = 4):
    transit_1_totals = sum(sum(transit_1, axis = 1),axis = 0)
    transit_2_totals = sum(sum(transit_2, axis = 1),axis = 0)
    time = arange(tmax)
    transit_1_pop = sum(locations[0:bar,0])
    transit_2_pop = sum(locations[bar:,0])
    transit_1_percent = transit_1_totals/transit_1_pop
    transit_2_percent = transit_2_totals/transit_2_pop
    if (transit_1_pop + transit_2_pop == n):
        fig, ax = plt.subplots()
        line1, = ax.plot(transit_1_percent, label='Group 1')
        line2, = ax.plot(transit_2_percent, label='Group 2')
        ax.legend(handles=[line1, line2])
        ax.set_xlabel("timestep")
        ax.set_ylabel("transit users (percent)")
        ax.set_ylim(0,0.6)
        if title:
            fig.savefig(title)
        plt.show()
    else:
        print("Population Sum Error")

In [ ]:
transitResults = transitMove(Network, relApp, tmax = 50, bar = 4)
transit_1 = transitResults['transit_1']
transit_2 = transitResults['transit_2']
locations = transitResults['locations']

plot_pop_compare(transit_1, transit_2, locations, tmax = 50, bar = 4)